## Model sold_quantity

The idea behind this notebook is build a model for the sold_quantity attribute of the products.

### Import libraries

In [1]:
import pandas as pd
import requests
import json
from sklearn.model_selection import train_test_split

import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

### Obtaining the data

If data has been already downloaded previously, load it uncommenting the following box:

In [2]:
#Uncomment to load csv file
productsDF = pd.read_csv("productosCatV2.csv", index_col=[0])

Otherwise download the data by categories. Full explanation for downloading the data is in notebook `Exploratory_Analysis`

In [3]:
#Uncomment for obtaining categories
#r =requests.get("https://api.mercadolibre.com/sites/MLA/categories")
#categories = r.json()

In [4]:
#url_base="https://api.mercadolibre.com/sites/MLA/search?category="

In [5]:
#productsList = []
#for category in categories:
#    for offset in range(0,500,50):
#        url = url_base + category["id"] + "&offset=" + str(offset)
#        r = requests.get(url)
#        result = r.json()
#        products= pd.DataFrame(result["results"])
#        products["parent_category_name"] = category["name"]
#        products["parent_category_id"] = category["id"]
#        productsList.append(products)
        
#productsDF = pd.concat(productsList)    

In [6]:
productsDF.head()

,id,site_id,title,seller,price,prices,sale_price,currency_id,available_quantity,sold_quantity,...,winner_item_id,melicoin,discounts,catalog_listing,differential_pricing,parent_category_name,parent_category_id,seller_contact,location,sale_terms
0,MLA821869592,MLA,Casco Integral Moto Vertigo Cosmic + Mascara +...,"{'id': 19876329, 'permalink': 'http://perfil.m...",4968.50,"{'id': 'MLA821869592', 'prices': [{'id': '155'...",NaN,ARS,150,500,...,NaN,NaN,NaN,NaN,NaN,Accesorios para Vehículos,MLA5725,NaN,NaN,NaN
1,MLA636709517,MLA,Cubre Alfombra Goma Pesada Universal 3 Piezas ...,"{'id': 173563910, 'permalink': 'http://perfil....",2159.99,"{'id': 'MLA636709517', 'prices': [{'id': '72',...",NaN,ARS,1,5000,...,NaN,NaN,NaN,NaN,NaN,Accesorios para Vehículos,MLA5725,NaN,NaN,NaN
2,MLA1102761474,MLA,Compresor De Aire Mini A Batería Portátil Klat...,"{'id': 671347056, 'permalink': 'http://perfil....",1999.00,"{'id': 'MLA1102761474', 'prices': [{'id': '48'...",NaN,ARS,456,3877,...,NaN,NaN,NaN,True,NaN,Accesorios para Vehículos,MLA5725,NaN,NaN,NaN
3,MLA610309020,MLA,Sensor De Estacionamiento * Colores Con Displa...,"{'id': 149089767, 'permalink': 'http://perfil....",1890.00,"{'id': 'MLA610309020', 'prices': [{'id': '5', ...",NaN,ARS,100,5000,...,NaN,NaN,NaN,NaN,NaN,Accesorios para Vehículos,MLA5725,NaN,NaN,NaN
4,MLA701195248,MLA,Cubre Volante De Cuero Legitimo Luca Tiziano C...,"{'id': 129081707, 'permalink': 'http://perfil....",1780.00,"{'id': 'MLA701195248', 'prices': [{'id': '5', ...",NaN,ARS,50000,500,...,NaN,NaN,NaN,NaN,NaN,Accesorios para Vehículos,MLA5725,NaN,NaN,NaN


In [7]:
productsDF.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15968 entries, 0 to 49
Data columns (total 44 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    15968 non-null  object 
 1   site_id               15968 non-null  object 
 2   title                 15968 non-null  object 
 3   seller                15968 non-null  object 
 4   price                 15892 non-null  float64
 5   prices                15968 non-null  object 
 6   sale_price            0 non-null      float64
 7   currency_id           15892 non-null  object 
 8   available_quantity    15968 non-null  int64  
 9   sold_quantity         15968 non-null  int64  
 10  buying_mode           15968 non-null  object 
 11  listing_type_id       15968 non-null  object 
 12  stop_time             15968 non-null  object 
 13  condition             15468 non-null  object 
 14  permalink             15968 non-null  object 
 15  thumbnail             

### Analyzing the data

We will work with the downloaded data, the target variable of interest is `sold_quantity`. The first step is usually to load (or download) the data which we have already done.  The second step is to split data into train, test and validate in order to do those operations. For that we will make use sklearn library.

#### Pre-proccesing of data

In [8]:
# Remove rows with missing target
productsDF.dropna(axis=0, subset=['sold_quantity'], inplace=True)
#sold_quantity is the target variable
y = productsDF.sold_quantity

To start with a simpler analysis I'm going to keep only numerical predictors:

In [9]:
numPredictors = productsDF.drop(['sold_quantity'], axis=1)
X = numPredictors.select_dtypes(exclude=['object'])

In [10]:
X.head()

,price,sale_price,available_quantity,accepts_mercadopago,original_price,official_store_id,order_backend,use_thumbnail_id,offer_score,offer_share,match_score,winner_item_id,melicoin,discounts
0,4968.50,NaN,150,True,5230.0,NaN,1,False,NaN,NaN,NaN,NaN,NaN,NaN
1,2159.99,NaN,1,True,NaN,NaN,2,False,NaN,NaN,NaN,NaN,NaN,NaN
2,1999.00,NaN,456,True,2399.0,1825.0,3,True,NaN,NaN,NaN,NaN,NaN,NaN
3,1890.00,NaN,100,True,NaN,NaN,4,False,NaN,NaN,NaN,NaN,NaN,NaN
4,1780.00,NaN,50000,True,NaN,NaN,5,True,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15968 entries, 0 to 49
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   price                15892 non-null  float64
 1   sale_price           0 non-null      float64
 2   available_quantity   15968 non-null  int64  
 3   accepts_mercadopago  15968 non-null  bool   
 4   original_price       2937 non-null   float64
 5   official_store_id    2434 non-null   float64
 6   order_backend        15968 non-null  int64  
 7   use_thumbnail_id     15968 non-null  bool   
 8   offer_score          0 non-null      float64
 9   offer_share          0 non-null      float64
 10  match_score          0 non-null      float64
 11  winner_item_id       0 non-null      float64
 12  melicoin             0 non-null      float64
 13  discounts            0 non-null      float64
dtypes: bool(2), float64(10), int64(2)
memory usage: 1.6 MB


I'm going to drop some columns:

In [12]:
columnsToDrop = ["sale_price","order_backend", "use_thumbnail_id", "offer_score", "offer_share", "match_score", "winner_item_id", "melicoin", "discounts"]

In [13]:
X = X.drop(columnsToDrop, axis=1)

In [14]:
X["accepts_mercadopago"] = X["accepts_mercadopago"].astype(float)   

In [15]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15968 entries, 0 to 49
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   price                15892 non-null  float64
 1   available_quantity   15968 non-null  int64  
 2   accepts_mercadopago  15968 non-null  float64
 3   original_price       2937 non-null   float64
 4   official_store_id    2434 non-null   float64
dtypes: float64(4), int64(1)
memory usage: 748.5 KB


And now I'm going to split the dataset:

In [16]:
# Divide data into training and validation subsets
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.9, test_size=0.1,random_state=0)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, train_size=0.9, test_size=0.1,random_state=0)

As I'm going to try out different approaches to modeling I'm going to define a fucntion to compare the results. This function return the mean absolute error (MAE) froma random forest model: 

In [17]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

In [18]:
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=10, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

### Modelling stage

#### First approach: Replacing missing numeric values with simple imputer

I am going to use SimplerImputer from sklearn to replace missing values with a mean value.

First we'll look for the columns with missing values:

In [19]:
colsWMissing = [col for col in X_train.columns
                     if X_train[col].isnull().any()]

In [20]:
colsWMissing

['price', 'original_price', 'official_store_id']

In [21]:
colsWMissingCount = (X_train.isnull().sum())
print(colsWMissingCount[colsWMissingCount > 0])

price                   66
original_price       10600
official_store_id    10962
dtype: int64


In [22]:
from sklearn.impute import SimpleImputer

# Imputation
my_imputer = SimpleImputer()
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(my_imputer.transform(X_valid))

In [23]:
print(imputed_X_train.columns)

RangeIndex(start=0, stop=5, step=1)


In [24]:
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns

In [25]:
print("MAE dropping categorical values and using imputer for numerical values:")
print(score_dataset(imputed_X_train, imputed_X_valid, y_train, y_valid))

MAE dropping categorical values and using imputer for numerical values:
484.7637672971337


In [26]:
# Preprocess test data
final_X_test = pd.DataFrame(my_imputer.transform(X_test))
final_X_test.columns = X_test.columns

# Define and fit model
model = RandomForestRegressor(n_estimators=100, random_state=0)
model.fit(imputed_X_train, y_train)

# Obtain test predictions
preds_test = model.predict(final_X_test)

In [27]:
print("MAE for test set:")
print(mean_absolute_error(y_test, preds_test))

MAE for test set:
512.1095294531239


#### Second approach: working with the categorical variables and numerical values

In [28]:
X = productsDF.drop(['sold_quantity'], axis=1)

In [29]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15968 entries, 0 to 49
Data columns (total 43 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    15968 non-null  object 
 1   site_id               15968 non-null  object 
 2   title                 15968 non-null  object 
 3   seller                15968 non-null  object 
 4   price                 15892 non-null  float64
 5   prices                15968 non-null  object 
 6   sale_price            0 non-null      float64
 7   currency_id           15892 non-null  object 
 8   available_quantity    15968 non-null  int64  
 9   buying_mode           15968 non-null  object 
 10  listing_type_id       15968 non-null  object 
 11  stop_time             15968 non-null  object 
 12  condition             15468 non-null  object 
 13  permalink             15968 non-null  object 
 14  thumbnail             15968 non-null  object 
 15  thumbnail_id          

For a simpler analysis I'm going to drop some of the columns:

In [30]:
columnsToDrop = ["site_id", "sale_price","order_backend", "use_thumbnail_id", "offer_score", "offer_share", "match_score", "winner_item_id", "melicoin", "discounts"]

In [31]:
X = X.drop(columnsToDrop, axis=1)

In [32]:
X["accepts_mercadopago"] = X["accepts_mercadopago"].astype(float)   

In [33]:
# Divide data into training, testing and validation subsets
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.9, test_size=0.1,random_state=0)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, train_size=0.9, test_size=0.1,random_state=0)

I am interested in knowing possible values of some categorical variables:

In [34]:
for cname in X_train.columns:
    if X_train[cname].dtype == "object":
        print(cname, X_train[cname].nunique())

id 12155
title 11932
seller 4256
prices 12155
currency_id 2
buying_mode 2
listing_type_id 5
stop_time 3997
condition 3
permalink 12135
thumbnail 11903
thumbnail_id 11879
installments 5710
address 692
shipping 200
seller_address 689
attributes 11730
category_id 1222
domain_id 1051
catalog_product_id 5143
tags 1026
catalog_listing 1
differential_pricing 11
parent_category_name 32
parent_category_id 32
seller_contact 367
location 925
sale_terms 16


I'm going to use an encoder but only in some of the categories, particularly up to those with len < 33 because the variables related to categories have lenght 32

In [35]:
categoricalCols = [cname for cname in X_train.columns if X_train[cname].nunique() < 33 and 
                        X_train[cname].dtype == "object"]

# Select numerical columns
numericalCols = [cname for cname in X_train.columns if X_train[cname].dtype in ['int64', 'float64']]

In [36]:
categoricalCols

['currency_id',
 'buying_mode',
 'listing_type_id',
 'condition',
 'catalog_listing',
 'differential_pricing',
 'parent_category_name',
 'parent_category_id',
 'sale_terms']

In [37]:
productsDF["buying_mode"].unique()

array(['buy_it_now', 'classified'], dtype=object)

In [38]:
productsDF["listing_type_id"].unique()

array(['gold_special', 'gold_pro', 'gold_premium', 'gold', 'bronze'],
      dtype=object)

In [39]:
productsDF["condition"].unique()

array(['new', 'used', 'not_specified', nan], dtype=object)

In [40]:
productsDF["differential_pricing"].unique()

array([nan, "{'id': 33678187}", "{'id': 35713320}", "{'id': 35713278}",
       "{'id': 35713279}", "{'id': 35713248}", "{'id': 35713268}",
       "{'id': 35713341}", "{'id': 35713287}", "{'id': 35713324}",
       "{'id': 35713291}", "{'id': 35713323}", "{'id': 35713326}"],
      dtype=object)

In [41]:
productsDF["sale_terms"].unique()

array([nan,
       "[{'name': 'Modalidad de precio', 'value_id': '6178426', 'value_name': 'Solicitar presupuesto', 'value_struct': None, 'values': [{'id': '6178426', 'name': 'Solicitar presupuesto', 'struct': None}], 'id': 'PRICE_MODALITY'}]",
       "[{'values': [{'id': '6178426', 'name': 'Solicitar presupuesto', 'struct': None}], 'id': 'PRICE_MODALITY', 'name': 'Modalidad de precio', 'value_id': '6178426', 'value_name': 'Solicitar presupuesto', 'value_struct': None}]",
       "[{'id': 'PRICE_MODALITY', 'name': 'Modalidad de precio', 'value_id': '6178426', 'value_name': 'Solicitar presupuesto', 'value_struct': None, 'values': [{'id': '6178426', 'name': 'Solicitar presupuesto', 'struct': None}]}]",
       "[{'value_name': 'Solicitar presupuesto', 'value_struct': None, 'values': [{'id': '6178426', 'name': 'Solicitar presupuesto', 'struct': None}], 'id': 'PRICE_MODALITY', 'name': 'Modalidad de precio', 'value_id': '6178426'}]",
       "[{'value_struct': None, 'values': [{'id': '6178426',

In [42]:
categoricalCols

['currency_id',
 'buying_mode',
 'listing_type_id',
 'condition',
 'catalog_listing',
 'differential_pricing',
 'parent_category_name',
 'parent_category_id',
 'sale_terms']

In [43]:
categoricalCols.remove('sale_terms')
categoricalCols.remove('currency_id')
categoricalCols.remove('buying_mode')

Columns of interest:

In [44]:
interstCols = categoricalCols + numericalCols
print(interstCols)

['listing_type_id', 'condition', 'catalog_listing', 'differential_pricing', 'parent_category_name', 'parent_category_id', 'price', 'available_quantity', 'accepts_mercadopago', 'original_price', 'official_store_id']


In [45]:
# Keep selected columns only
X_train = X_train[interstCols].copy()
X_valid = X_valid[interstCols].copy()

Use One-Hot Encoding to transform categorical values

In [46]:
# Get list of categorical variables
s = (X_train.dtypes == 'object')
objectCols = list(s[s].index)

In [47]:
print("Categorical variables:")
print(objectCols)

Categorical variables:
['listing_type_id', 'condition', 'catalog_listing', 'differential_pricing', 'parent_category_name', 'parent_category_id']


In [48]:
from sklearn.preprocessing import OneHotEncoder

# Apply one-hot encoder to each column with categorical data
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train[objectCols]))
OH_cols_valid = pd.DataFrame(OH_encoder.transform(X_valid[objectCols]))

# One-hot encoding removed index; put it back
OH_cols_train.index = X_train.index
OH_cols_valid.index = X_valid.index

# Remove categorical columns (will replace with one-hot encoding)
num_X_train = X_train.drop(objectCols, axis=1)
num_X_valid = X_valid.drop(objectCols, axis=1)


Use simple imputer for numerical values:

In [49]:
num_X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12933 entries, 25 to 3
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   price                12867 non-null  float64
 1   available_quantity   12933 non-null  int64  
 2   accepts_mercadopago  12933 non-null  float64
 3   original_price       2333 non-null   float64
 4   official_store_id    1971 non-null   float64
dtypes: float64(4), int64(1)
memory usage: 606.2 KB


In [50]:
# Imputation
my_imputer = SimpleImputer()
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train[numericalCols]))
imputed_X_valid = pd.DataFrame(my_imputer.transform(X_valid[numericalCols]))
imputed_X_train.index = X_train.index
imputed_X_valid.index = X_valid.index

In [51]:
imputed_X_valid.columns

RangeIndex(start=0, stop=5, step=1)

In [52]:
# Add one-hot encoded columns to numerical features
OH_X_train = pd.concat([imputed_X_train, OH_cols_train], axis=1)
OH_X_valid = pd.concat([imputed_X_valid, OH_cols_valid], axis=1)

In [53]:
print("MAE for approach One-Hot Encoding + imputer with mean:") 
print(score_dataset(OH_X_train, OH_X_valid, y_train, y_valid))

MAE for approach One-Hot Encoding + imputer with mean:
509.46230125998363


In [54]:
# Preprocess test data
OH_cols_test = pd.DataFrame(OH_encoder.transform(X_test[objectCols]))
imputed_X_test = pd.DataFrame(my_imputer.transform(X_test[numericalCols]))

OH_cols_test.index = X_test.index
imputed_X_test.index = X_test.index
OH_X_test = pd.concat([imputed_X_test, OH_cols_test], axis=1)

In [55]:
OH_X_test.shape

(1597, 92)

In [56]:
X_test.shape

(1597, 33)

In [57]:
OH_X_train.shape

(12933, 92)

In [58]:
y_train.shape

(12933,)

In [59]:
# Define and fit model
model = RandomForestRegressor(n_estimators=100, random_state=0)
model.fit(OH_X_train, y_train)

RandomForestRegressor(random_state=0)

In [60]:
# Obtain test predictions
preds_test = model.predict(OH_X_test)

In [61]:
print("MAE for test set:")
print(mean_absolute_error(y_test, preds_test))

MAE for test set:
517.2996631477945
